In [2]:
import pandas as pd
import numpy as np

In [3]:
# Set option to display all columns
pd.set_option('display.max_columns', None)

In [4]:
df = pd.read_parquet('data/sample.parquet')
df

,time,value,field,robot_id,run_uuid,sensor_type
0,2022-11-23T20:40:00.005Z,821.780800,x,1,8.910096e+18,encoder
1,2022-11-23T20:40:00.017Z,821.821700,x,1,8.910096e+18,encoder
2,2022-11-23T20:40:00.029Z,821.850700,x,1,8.910096e+18,encoder
3,2022-11-23T20:40:00.041Z,821.896400,x,1,8.910096e+18,encoder
4,2022-11-23T20:40:00.053Z,821.957300,x,1,8.910096e+18,encoder
...,...,...,...,...,...,...
1546586,2022-11-23T20:41:17.59Z,-85.692373,fx,1,1.240519e+19,load_cell
1546587,2022-11-23T20:41:17.6Z,-87.231436,fx,1,1.240519e+19,load_cell
1546588,2022-11-23T20:41:17.61Z,-85.649405,fx,1,1.240519e+19,load_cell
1546589,2022-11-23T20:41:17.62Z,-86.430655,fx,1,1.240519e+19,load_cell


## Sort the rows based on the time to make sure that the rows are stored sequentialy

In [5]:
df = df.sort_values('time')
df

,time,value,field,robot_id,run_uuid,sensor_type
878828,2022-11-23T20:40:00.001Z,716.528276,fy,1,7.582293e+18,load_cell
938829,2022-11-23T20:40:00.001Z,-1547.340972,fz,1,7.582293e+18,load_cell
818827,2022-11-23T20:40:00.001Z,-1192.046953,fx,1,7.582293e+18,load_cell
1118830,2022-11-23T20:40:00.003Z,84.484822,fz,2,7.582293e+18,load_cell
1058830,2022-11-23T20:40:00.003Z,489.207227,fy,2,7.582293e+18,load_cell
...,...,...,...,...,...,...
550011,2022-11-23T20:49:59.9Z,1000.769000,y,2,7.582293e+18,encoder
600012,2022-11-23T20:49:59.9Z,-771.632000,z,2,7.582293e+18,encoder
549936,2022-11-23T20:49:59Z,1000.769000,y,2,7.582293e+18,encoder
599937,2022-11-23T20:49:59Z,-771.632000,z,2,7.582293e+18,encoder


# 2.1 Preprocess and Clean

## A method to check the data type of each column. If all column types are as expected this method pass otherwise it shows the columns with a mismatch

In [6]:
def check_data_types(df, expected_types):
    """
    Check if the DataFrame's columns have the expected data types.

    :param df: pandas DataFrame to check
    :param expected_types: Dictionary of expected data types {column_name: expected_type}
    :return: None if types match, otherwise raises ValueError with mismatch details
    """
    for column, expected_type in expected_types.items():
        if column in df.columns:
            if not pd.api.types.is_dtype_equal(df[column].dtype, expected_type):
                raise ValueError(f"Data type mismatch in column '{column}'. Expected {expected_type}, found {df[column].dtype}")
        else:
            raise ValueError(f"Column '{column}' not found in DataFrame")
    print("All column data types are as expected.")

In [7]:
# Define the expected types
expected_types = {
    'time': 'object',
    'value': 'float64',
    'field': 'object',
    'robot_id': 'int64',
    'run_uuid': 'float64',
    'sensor_type': 'object'
}

# Check data types
check_data_types(df, expected_types)

All column data types are as expected.


# 2.2 Convert timeseries to a wide format

## Here, I split the data based on each part (run_uuid)

In [8]:
# Identify Unique run_uuid Values
unique_run_uuids = df['run_uuid'].unique()

# Split the DataFrame
dfs = {}
for run_uuid in unique_run_uuids:
    # Create a separate DataFrame for each run_uuid
    dfs[run_uuid] = df[df['run_uuid'] == run_uuid]

## For each part, I concatenated the field an robot_id and then used pivot to have them in one row and convert them to a wide format.

In [9]:
for run_uuid, temp_df in dfs.items():
    # Explicitly create a copy of the DataFrame slice
    temp_df = temp_df.copy()

    # Combine 'field' and 'robot_id'
    temp_df['field_robot'] = temp_df['field'] + '_' + temp_df['robot_id'].astype(str)

    # Pivot the DataFrame
    pivot_df = temp_df.pivot_table(index='time', columns='field_robot', values='value', aggfunc='first')
    
    # Replace the original DataFrame in the dictionary with the pivoted one
    dfs[run_uuid] = pivot_df

## Here for each part, there is one dataframe. They have all possible columns, but for some of the columns there is no data. So I keep the columns and add '0' for such columns.

In [10]:
required_columns = ['fx_1', 'fx_2', 'fy_1', 'fy_2', 'fz_1', 'fz_2', 'x_1', 'x_2', 'y_1', 'y_2', 'z_1', 'z_2']

for run_uuid, df in dfs.items():
    # Identify missing columns
    missing_columns = [col for col in required_columns if col not in df.columns]

    # Add missing columns with default value of 0
    for col in missing_columns:
        df[col] = 0

    # Optionally, reorder columns to match the order in required_columns
    df = df.reindex(columns=required_columns)

    # Update the DataFrame in the dictionary
    dfs[run_uuid] = df

## After converting the timeseries to individual features, there are many gaps in the data. So the dataframe has null values. I used interpolation to fill the null values. Since the data is a time series and there is some form of continutiy, interpolation could be used here. It estimated the null values using existing values.

In [11]:
for run_uuid, df in dfs.items():
    # Interpolate to fill missing values
    df_interpolated = df.interpolate()

    # Forward fill or backward fill to handle remaining missing values
    df_filled = df_interpolated.fillna(method='bfill')

    # Update the DataFrame in the dictionary
    dfs[run_uuid] = df_filled

# 2.3 Include Engineered/Calculated Features

## The velocity, acceleration, total velocity, total acceleration and total force values are calculated here. I used the current position of the robot, the previous position and the time difference between them to calcualte teh velocity. The way that the velocity and acceleration might need modifications but the logic works fine here.

In [12]:
for run_uuid, df in dfs.items():
    # Ensure the index is a datetime index
    df.index = pd.to_datetime(df.index)

    # Calculate the time differences in seconds
    df['time_diff'] = df.index.to_series().diff().dt.total_seconds()

    # Calculate velocities
    for axis in ['x', 'y', 'z']:
        for robot_id in [1, 2]:
            pos_col = f'{axis}_{robot_id}'
            vel_col = f'v{axis}_{robot_id}'
            df[vel_col] = df[pos_col].diff() / df['time_diff']

    # Calculate accelerations
    for axis in ['x', 'y', 'z']:
        for robot_id in [1, 2]:
            vel_col = f'v{axis}_{robot_id}'
            acc_col = f'a{axis}_{robot_id}'
            df[acc_col] = df[vel_col].diff() / df['time_diff']

    # Calculate total velocities and accelerations
    for robot_id in [1, 2]:
        df[f'v{robot_id}'] = (df[f'vx_{robot_id}']**2 + df[f'vy_{robot_id}']**2 + df[f'vz_{robot_id}']**2)**0.5
        df[f'a{robot_id}'] = (df[f'ax_{robot_id}']**2 + df[f'ay_{robot_id}']**2 + df[f'az_{robot_id}']**2)**0.5

    # Calculate total forces
    for robot_id in [1, 2]:
        df[f'f{robot_id}'] = abs(df[f'fx_{robot_id}']) + abs(df[f'fy_{robot_id}']) + abs(df[f'fz_{robot_id}'])

    # Update the DataFrame in the dictionary
    dfs[run_uuid] = df

# 2.4 Calculate Runtime Statistics

## Runtime Statistics is produced for each part and stored in a pandas dataframe. This dataframe is then saved in an csv file.

In [13]:
results_list = []

for run_uuid, df in dfs.items():
    # Ensure the index is a datetime index
    df.index = pd.to_datetime(df.index)

    # Start and stop times
    start_time = df.index[0]
    stop_time = df.index[-1]

    # Total runtime
    total_runtime = stop_time - start_time

    # Initialize total distance for each robot
    total_distance = {1: 0, 2: 0}

    # Calculate total distance traveled for each robot
    for robot_id in [1, 2]:
        # Calculate differences in x, y, z coordinates
        dx = df[f'x_{robot_id}'].diff().fillna(0)
        dy = df[f'y_{robot_id}'].diff().fillna(0)
        dz = df[f'z_{robot_id}'].diff().fillna(0)

        # Calculate distance for each step and sum
        total_distance[robot_id] = np.sqrt(dx**2 + dy**2 + dz**2).sum()

    # Create a dictionary of the results and add it to the list
    results_dict = {
        'run_uuid': run_uuid,
        'start_time': start_time,
        'stop_time': stop_time,
        'total_runtime': total_runtime,
        'total_distance_1': total_distance[1],
        'total_distance_2': total_distance[2]
    }
    results_list.append(results_dict)

# Convert the list of dictionaries to a DataFrame
results_df = pd.DataFrame(results_list)

In [14]:
# Specify the file name for the CSV file
csv_file_name = 'results.csv'

# Write the DataFrame to a CSV file
results_df.to_csv(csv_file_name, index=False)